# Kernel methods 

### Import des fonctions


In [158]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy.linalg import solve,lstsq
from scipy.stats import uniform

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from itertools import compress
import numexpr as ne

In [2]:
X_0=pd.read_csv("Data/Xtr0_mat100.csv",sep=' ',header=None)
X_1=pd.read_csv("Data/Xtr1_mat100.csv",sep=' ',header=None)
X_2=pd.read_csv("Data/Xtr2_mat100.csv",sep=' ',header=None)


Y_0=pd.read_csv("Data/Ytr0.csv",sep=',')
Y_1=pd.read_csv("Data/Ytr1.csv",sep=',')
Y_2=pd.read_csv("Data/Ytr2.csv",sep=',')

X_0

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.010870,0.010870,0.043478,0.010870,0.021739,0.021739,0.000000,0.010870,0.010870,0.010870,...,0.000000,0.032609,0.01087,0.010870,0.010870,0.000000,0.000000,0.010870,0.000000,0.01087
1,0.000000,0.000000,0.010870,0.010870,0.000000,0.010870,0.010870,0.000000,0.000000,0.000000,...,0.010870,0.010870,0.01087,0.021739,0.000000,0.010870,0.021739,0.032609,0.000000,0.00000
2,0.021739,0.010870,0.021739,0.000000,0.000000,0.000000,0.010870,0.010870,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.010870,0.000000,0.021739,0.000000,0.021739,0.021739,0.01087
3,0.021739,0.010870,0.043478,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,0.010870,...,0.021739,0.000000,0.00000,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,0.01087
4,0.021739,0.010870,0.010870,0.010870,0.010870,0.010870,0.032609,0.000000,0.010870,0.043478,...,0.010870,0.010870,0.00000,0.000000,0.010870,0.010870,0.032609,0.010870,0.021739,0.01087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.000000,0.000000,0.000000,0.010870,0.000000,0.010870,0.000000,0.021739,0.010870,0.032609,...,0.000000,0.010870,0.01087,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.00000
1996,0.010870,0.021739,0.000000,0.000000,0.000000,0.021739,0.010870,0.010870,0.021739,0.000000,...,0.021739,0.010870,0.00000,0.010870,0.000000,0.000000,0.000000,0.000000,0.010870,0.01087
1997,0.010870,0.000000,0.000000,0.000000,0.000000,0.010870,0.010870,0.010870,0.021739,0.000000,...,0.010870,0.010870,0.00000,0.000000,0.021739,0.000000,0.032609,0.032609,0.000000,0.00000
1998,0.010870,0.010870,0.000000,0.000000,0.021739,0.000000,0.010870,0.021739,0.000000,0.032609,...,0.032609,0.000000,0.00000,0.010870,0.000000,0.010870,0.000000,0.000000,0.000000,0.01087


In [3]:
print(Y_0["Bound"].value_counts())
print(Y_1["Bound"].value_counts())
print(Y_2["Bound"].value_counts())

0    1038
1     962
Name: Bound, dtype: int64
1    1001
0     999
Name: Bound, dtype: int64
0    1003
1     997
Name: Bound, dtype: int64


Les classes sont plutôt équilibrées 

In [4]:
Y_0train=np.where(Y_0["Bound"]==0,-1,Y_0["Bound"])
Y_1train=np.where(Y_1["Bound"]==0,-1,Y_1["Bound"])
Y_2train=np.where(Y_2["Bound"]==0,-1,Y_2["Bound"])

X_0train=X_0.to_numpy()
X_1train=X_1.to_numpy()
X_2train=X_2.to_numpy()
print(X_0train.shape)

[-1  1  1 ... -1 -1 -1]


## Ridge regression ( équivalent à Kernel ridge régression avec noyau linéaire) 
On ajoute un colonne de biais 

In [5]:
X_0train_forRR=np.hstack((np.ones((X_0train.shape[0],1)),X_0train))
X_1train_forRR=np.hstack((np.ones((X_0train.shape[0],1)),X_0train))
X_2train_forRR=np.hstack((np.ones((X_0train.shape[0],1)),X_0train))

### Solver ridge Regression 
Même si les classes sont équilibrées je laisse la possibilité de modifier les poids. Enfin pas vraiment puis que le seul poid que je met quand on souhaite en mettre c'est le poid correspondant au poids de classes dans le set qui sera entraîné.
J'utilise lstq pour résoudre le système lineaire car plus stable si la matrice est mal conditionné/pas inversible) 

In [6]:
def Ridge_Regresssion(X_train,y_train,lbd,weight=False):
    n=y_train.shape[0]
    d=X_train.shape[1]
    if not(weight):
        A=(X_train.T.dot(X_train))/n+lbd*np.eye(d)
        B=X_train.T.dot(y_train)/n
        theta=lstsq(A,B)[0]
        return theta
    else:
        w1=(y_train==1).mean()
        w0=1-w1
        w=np.where(y_train==1,w1,w0)
        W=np.diag(w)
        A=(X_train.T.dot(W.dot(X_train)))/n+lbd*np.eye(d)
        B=X_train.T.dot(W.dot(y_train))/n
        theta=lstq(A,B)[0]
        return theta


 ### Création de l'estimateur Ridge regression compatible avec l'API de scikit learn

In [17]:
class LinearRR(BaseEstimator,ClassifierMixin):
    def __init__(self,lbd=1,weight=False):
        self.lbd=lbd
        self.weight=weight
        
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        
        self.theta=Ridge_Regresssion(X,y,self.lbd,self.weight)
        return self
    def decision_function(self,X):
        return X.dot(self.theta)
        
    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
   
    def get_params(self, deep=True):
    
        return {"lbd": self.lbd,"weight":self.weight}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

### Fonction permettant d'afficher les n meilleurs résultats avec les paramètres associées d'une grid search dans un tableau (dataframe)

In [137]:
def présentation_résultat(search,n):
    mask=search.cv_results_['rank_test_score']<=n
    params=list(compress(search.cv_results_['params'], list(mask)))
    mean_test_score=search.cv_results_['mean_test_score'][mask]
    a={}
    for i in range(mean_test_score.size):
        k=''
        for key, value in params[i].items():
            k+=" "+key+" "+str(value)
        a.update({k:mean_test_score[i]})
        sortedDict = sorted(a.items(), key=lambda x: x[1],reverse=True)
    l=[]
    for i in sortedDict:
        u=i[0].split(sep=' ')
        del(u[0])
        lp=[]
        for j in u[1::2]:
            lp.append(j)
        lp.append(i[1])
        l.append(lp)
    head=list(params[0].keys())+["mean_test_score"]

    return(pd.DataFrame(l,columns=head))
        
    
    
    
    

## Première recherche des hyperparamètres pour chaque train set 
Je crée un pipeline composé d'un transformer de données et de la ridgre regression. Dans la manière dont j'ai défini la pipeline le transformer en question est *StandardScaler()* qui standardise les données ((X-mean)/std)). Attention cela standardise les données selon la mean et std du train set et donc lors d'une prédiction le test/validation set sera standardisé par rapport à la mean et std du training set. 
En soit la manière dont j'ai défini le tranformeur n'a pas d'importance car dans la gridSearch je teste d'autres transformeur à certain moment(mais fallait bien en mettre pour faire la pipeline), il y a même une option *"passthrough"* pour la première étape de la pipeline qui veut dire tout simplement passer à l'étape suivante et donc ne pas transfomrer mes données.


In [141]:
RR=Pipeline([("preprocess",StandardScaler()),("linearrr",LinearRR())])
print(RR)

hps0={"linearrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"linearrr__weight":[False,True],
     "preprocess":["passthrough",StandardScaler(),RobustScaler(),MinMaxScaler()]}
searchLRR0= GridSearchCV(RR,hps0,scoring="accuracy",cv=5,verbose=1)
searchLRR0.fit(X_0train_forRR, Y_0train)

hps1={"linearrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"linearrr__weight":[False,True],
     "preprocess":["passthrough",StandardScaler(),RobustScaler(),MinMaxScaler()]}
searchLRR1= GridSearchCV(RR,hps1,scoring="accuracy",cv=5,verbose=1)
searchLRR1.fit(X_1train_forRR, Y_1train)


hps2={"linearrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"linearrr__weight":[False,True],
     "preprocess":["passthrough",StandardScaler(),RobustScaler(),MinMaxScaler()]}
searchLRR2= GridSearchCV(RR,hps2,scoring="accuracy",cv=5,verbose=1)
searchLRR2.fit(X_2train_forRR, Y_2train)


Pipeline(steps=[('preprocess', StandardScaler()), ('linearrr', LinearRR())])
Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:   13.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:   13.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:   14.0s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess', StandardScaler()),
                                       ('linearrr', LinearRR())]),
             param_grid={'linearrr__lbd': [1e-06, 1e-05, 0.0001, 0.001, 0.01,
                                           0.1, 1, 10],
                         'linearrr__weight': [False, True],
                         'preprocess': ['passthrough', StandardScaler(),
                                        RobustScaler(), MinMaxScaler()]},
             scoring='accuracy', verbose=1)

## Affichage des résulats 

In [138]:
print(présentation_résultat(searchLRR0,10))
print(présentation_résultat(searchLRR1,10))
print(présentation_résultat(searchLRR2,10))

   linearrr__lbd linearrr__weight        preprocess  mean_test_score
0              1             True  StandardScaler()           0.5975
1              1            False  StandardScaler()           0.5945
2            0.1             True  StandardScaler()           0.5940
3           0.01            False    MinMaxScaler()           0.5930
4            0.1            False  StandardScaler()           0.5905
5          0.001             True    MinMaxScaler()           0.5890
6           0.01            False  StandardScaler()           0.5885
7           0.01             True  StandardScaler()           0.5870
8          0.001             True  StandardScaler()           0.5865
9          1e-06             True  StandardScaler()           0.5860
10         1e-05             True  StandardScaler()           0.5860
11        0.0001             True  StandardScaler()           0.5860
12         0.001            False  StandardScaler()           0.5860
   linearrr__lbd linearrr__weight 

### Recherche plus fine dans les zones d'intérêts déterminés à l'aide la search précedente

Ici je fais une RandomizedGrid search car j'ai beaucoup plus d'inormations sur les zones d'intérêt où il faut chercher(notemment pour le lambda). Si je l'avais fait avant étant donné que j'avais aucun à priori sur la valeur de lambda la recherche au hasard aurait été plus difficile 

In [144]:

hps0={"linearrr__lbd":uniform(loc=0.005,scale=5),"linearrr__weight":[False,True]}
searchLRR0= RandomizedSearchCV(RR,hps0,n_iter=1000,scoring="accuracy",cv=5,verbose=1)
searchLRR0.fit(X_0train_forRR, Y_0train)

hps1={"linearrr__lbd":uniform(loc=0.05,scale=10),"linearrr__weight":[False,True],
     "preprocess":["passthrough",StandardScaler(),RobustScaler(),MinMaxScaler()]}
searchLRR1= RandomizedSearchCV(RR,hps1,n_iter=1000,scoring="accuracy",cv=5,verbose=1)
searchLRR1.fit(X_1train_forRR, Y_1train)


hps2={"linearrr__lbd":uniform(loc=0.005,scale=10),"linearrr__weight":[False,True],
     "preprocess":["passthrough"]}
searchLRR2= RandomizedSearchCV(RR,hps2,n_iter=1000,scoring="accuracy",cv=5,verbose=1)
searchLRR2.fit(X_2train_forRR, Y_2train)


Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 5000 out of 5000 | elapsed:  2.7min finished


Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 5000 out of 5000 | elapsed:  3.6min finished


Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 5000 out of 5000 | elapsed:  1.8min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocess', StandardScaler()),
                                             ('linearrr', LinearRR())]),
                   n_iter=1000,
                   param_distributions={'linearrr__lbd': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000258DC893B50>,
                                        'linearrr__weight': [False, True],
                                        'preprocess': ['passthrough']},
                   scoring='accuracy', verbose=1)

### Présentation des résultats 

In [149]:
print(présentation_résultat(searchLRR0,10))
print(présentation_résultat(searchLRR1,10))
print(présentation_résultat(searchLRR2,2))

         linearrr__lbd linearrr__weight  mean_test_score
0   0.6808368789557301             True           0.5995
1   0.6488895482849216             True           0.5995
2   0.6767049343351716             True           0.5995
3    0.621128518027827             True           0.5990
4    0.664829316929445             True           0.5990
5    0.622062086234566             True           0.5990
6    0.608279167880712             True           0.5990
7    0.639381675975868             True           0.5990
8     1.23353765953284            False           0.5990
9   1.2598755980184615            False           0.5985
10  1.2455362990019019            False           0.5985
          linearrr__lbd linearrr__weight      preprocess  mean_test_score
0     1.666125807871538             True  MinMaxScaler()           0.5165
1    1.6427502712439634             True  MinMaxScaler()           0.5165
2   0.14439490141905037            False  MinMaxScaler()           0.5165
3    1.6842109992352

### Recherche encore plus fine (sert pas trop à grand chose car on va garder ce modèle)

In [153]:
hps0={"linearrr__lbd":uniform(loc=0.5,scale=1.5),"linearrr__weight":[False,True]}
searchLRR0= RandomizedSearchCV(RR,hps0,n_iter=500,scoring="accuracy",cv=5,verbose=1)
searchLRR0.fit(X_0train_forRR, Y_0train)

hps1={"linearrr__lbd":uniform(loc=1.5,scale=4.5),"linearrr__weight":[True],
     "preprocess":[MinMaxScaler()]}
searchLRR1= RandomizedSearchCV(RR,hps1,n_iter=500,scoring="accuracy",cv=5,verbose=1)
searchLRR1.fit(X_1train_forRR, Y_1train)


hps2={"linearrr__lbd":uniform(loc=0.005,scale=0.1),"linearrr__weight":[False,True],
     "preprocess":["passthrough"]}
searchLRR2= RandomizedSearchCV(RR,hps2,n_iter=500,scoring="accuracy",cv=5,verbose=1)
searchLRR2.fit(X_2train_forRR, Y_2train)


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2500 out of 2500 | elapsed:  2.1min finished


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2500 out of 2500 | elapsed:  2.9min finished


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2500 out of 2500 | elapsed:  1.5min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocess', StandardScaler()),
                                             ('linearrr', LinearRR())]),
                   n_iter=500,
                   param_distributions={'linearrr__lbd': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000258D7D182E0>,
                                        'linearrr__weight': [False, True],
                                        'preprocess': ['passthrough']},
                   scoring='accuracy', verbose=1)

### Présentation des résultats 

In [254]:
print(présentation_résultat(searchLRR0,10))
print(présentation_résultat(searchLRR1,10))
print(présentation_résultat(searchLRR2,10))

         linearrr__lbd linearrr__weight  mean_test_score
0   0.6474957713207163             True           0.5995
1   0.6710958253318303             True           0.5995
2   0.6780076073848955             True           0.5995
3    0.684706500425291             True           0.5995
4   0.6566284985873325             True           0.5995
5   0.6527012855360701             True           0.5995
6   0.6332628536368978             True           0.5995
7    0.636330197042721             True           0.5995
8   1.3388173314288454            False           0.5990
9   1.2439881721589723            False           0.5990
10  0.7009797809890661             True           0.5990
11  0.6970090412156806             True           0.5990
12  0.6959957407336215             True           0.5990
13  0.6250852638590156             True           0.5990
14  0.6983219002757041             True           0.5990
15  0.6178297739883101             True           0.5990
16  0.6158370984442172         

# Kernel Ridge regression 

### Crétion du kernel de manière efficiente voir ce lien pour comprendre https://stackoverflow.com/questions/47271662/what-is-the-fastest-way-to-compute-an-rbf-kernel-in-python 
Mais ce qu'il y a retenir c'est que $\| x-y \|^2=\|x\|^2 + \|y\|^2 -2x^Ty$ et d'utiliser à son avantage le broadcast de Numpy. Le ne.evaluate est là pour rendre l'opération plus rapide mais l'opération qui est faite et bien "exp(-g*(A+B-2*C))" Avec les A B C g correspondant. D'ailleurs la propriété précédente nous permet de manière générale de calculer la matrice $K(x_i,y_i)$ pour $y_1,\dots,y_k ~~x_1,\dots,x_n$  $y_i,x_i \in R^d$ La kernel matrice et un cas particulier où $n=k$ et $x_i=y_i$ pour tout $i$.
Cette fonction est contruite juste après la fonction qui construit la kernel matrice. Elle nous sera très utile en espérant que vous aviez déjà deviné pourquoi, c'est pour  calculer $\left(f(z_1),\dots,f(z_k)\right)$ oû $f(z_j)=\sum_{i=1}^{n}\alpha_i K(z_j,x_i)$ 


In [194]:
def rbf_kernel(X_train,gamma):
    X_norm = np.sum(X_train ** 2, axis = -1)
    K = ne.evaluate('exp(-g * (A + B - 2 * C))', {
        'A' : X_norm[:,None],
        'B' : X_norm[None,:],
        'C' : np.dot(X_train, X_train.T),
        'g' : gamma,
    })
    return K

In [215]:
def K_rbf_kernel(X_test,X_train,gamma):
   
    Xtt_norm = np.sum(X_test ** 2, axis = -1)
    Xtr_norm = np.sum(X_train ** 2, axis = -1)
    K = ne.evaluate('exp(-g * (A + B - 2 * C))', {
        'A' : Xtt_norm[:,None],
        'B' : Xtr_norm[None,:],
        'C' : np.dot(X_test, X_train.T),
        'g' : gamma,
    })
    return K
    

### Création du solver Kernel ridge regresion

In [250]:
def Kernel_Ridge_Regression(X_train,y_train,lbd,weight,gamma):
    K=rbf_kernel(X_train,gamma)
    n=K.shape[0]
    if not(weight):
        A=(K+n*lbd*np.eye(n))
        alpha=lstsq(A,y_train)[0]
        return alpha
    else :
        w1=(y_train==1).mean()
        w0=1-w1
        w=np.where(y_train==1,w1,w0)
        Wr=np.diag(w**0.5)
        Wir=np.diag((1/w)**0.5)
        A=((Wr.dot(K.dot(Wr)))+n*lbd*np.eye(n)).dot(Wir)
        alpha=lstsq(A,Wr.dot(y_train))[0]
        return alpha
        
    

### Création de l'estimateur compatible avec l'Api de scikit 

In [251]:
class KernelRR(BaseEstimator,ClassifierMixin):
    def __init__(self,lbd=1,weight=False,gamma=1):
        self.lbd=lbd
        self.weight=weight
        self.gamma=gamma
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        self.Xtr=X
        self.alpha=Kernel_Ridge_Regression(X,y,self.lbd,self.weight,self.gamma)
        return self
    def decision_function(self,X):
        return K_rbf_kernel(X,self.Xtr,self.gamma).dot(self.alpha) 
        
    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
   
    def get_params(self, deep=True):
    
        return {"lbd": self.lbd,"weight":self.weight,"gamma":self.gamma}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

### Recherche des hyperparamètres pour KRR avec noyau gaussien 
Pas tester la transformation des données avant de cré la kernel matrices à faire 

In [252]:
KRR=KernelRR()
print(KRR)

hps0={"lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"weight":[False,True],
     "gamma":[0.001,0.01,0.1,1,10]}
searchKRR0= GridSearchCV(KRR,hps0,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR0.fit(X_0train, Y_0train)

hps1={"lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"weight":[False,True],
     "gamma":[0.001,0.01,0.1,1,10]}
searchKRR1= GridSearchCV(KRR,hps1,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR1.fit(X_1train, Y_1train)


hps2={"lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"weight":[False,True],
     "gamma":[0.001,0.01,0.1,1,10]}
searchKRR2= GridSearchCV(KRR,hps2,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR2.fit(X_2train, Y_2train)



KernelRR()
Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 12.0min finished


Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 19.7min finished


Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 12.4min finished


GridSearchCV(cv=5, estimator=KernelRR(), n_jobs=-1,
             param_grid={'gamma': [0.001, 0.01, 0.1, 1, 10],
                         'lbd': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10],
                         'weight': [False, True]},
             scoring='accuracy', verbose=1)

### Présentation des Résultats 
On remarque pour le training set 1 on a 10% en plus 20% en plus!!! pour le training set 2 et rien ne change pour le training set 0 par rapport au modèle linéraire. A Approfondir

In [253]:
print(présentation_résultat(searchKRR0,10))
print(présentation_résultat(searchKRR1,10))
print(présentation_résultat(searchKRR2,10))

  gamma     lbd weight  mean_test_score
0     1   1e-06  False           0.5945
1  0.01   1e-06  False           0.5935
2    10  0.0001  False           0.5935
3   0.1   1e-05  False           0.5930
4     1  0.0001  False           0.5930
5     1   1e-05  False           0.5910
6     1   1e-05   True           0.5900
7    10  0.0001   True           0.5890
8    10   0.001  False           0.5885
9     1   1e-06   True           0.5865
  gamma     lbd weight  mean_test_score
0     1   1e-06  False           0.6025
1    10  0.0001  False           0.5995
2    10  0.0001   True           0.5980
3     1   1e-06   True           0.5955
4    10   1e-05   True           0.5930
5    10   0.001  False           0.5825
6    10   1e-05  False           0.5805
7     1   1e-05  False           0.5785
8     1  0.0001  False           0.5765
9    10   0.001   True           0.5765
  gamma     lbd weight  mean_test_score
0    10  0.0001  False           0.7050
1    10  0.0001   True           0.7015
